In [7]:
# importing Qiskit
from qiskit import IBMQ, Aer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, transpile, assemble
import time

# import basic plot tools
from qiskit.visualization import plot_histogram
from qiskit_textbook.tools import simon_oracle

backendObject = Aer.get_backend('qasm_simulator')


In [34]:
def endianNotation(numberOfQubits) :
    endianRep = "{0:b}".format(numberOfQubits)[::-1]
    bitMapLength = len(endianRep)
    zeroPadding = '0' * (numberOfQubits - bitMapLength)
    endianRep = endianRep + zeroPadding
    print("Bitmap: " + endianRep)
    return endianRep

In [35]:
def bdotz(b, z):
    accum = 0
    for i in range(len(b)):
        accum += int(b[i]) * int(z[i])
    return (accum % 2)


In [36]:
def buildSimonCircuit(numberOfQubits) :
    simon_circuit = QuantumCircuit(numberOfQubits*2, numberOfQubits)
    
    # Apply Hadamard gates before querying the oracle
    simon_circuit.h(range(numberOfQubits))    
    
    # Apply barrier for visual separation
    simon_circuit.barrier()

    simon_circuit += simon_oracle(endianNotation(numberOfQubits))

    # Apply barrier for visual separation
    simon_circuit.barrier()

    # Apply Hadamard gates to the input register
    simon_circuit.h(range(numberOfQubits))

    # Measure qubits
    simon_circuit.measure(range(numberOfQubits), range(numberOfQubits))
    
    return simon_circuit

In [39]:
def runSimonAlgorithm(numberOfQubits) :
    simonCircuit = buildSimonCircuit(numberOfQubits)
    shots = 1024
    quantumObj = assemble(simonCircuit, shots=shots)
    timeBefore = time.perf_counter()
    results = backendObject.run(quantumObj).result()
    counts = results.get_counts()
    timeAfter = time.perf_counter()
    totalElapsedTime = timeAfter - timeBefore   
    formattedTotalElapsedTime = "{:.4f}".format(totalElapsedTime)  
    print("[" + str(numberOfQubits) + "] Time: " + str(formattedTotalElapsedTime))

    print(counts)
    plot_histogram(counts)
    b = endianNotation(numberOfQubits)  
    for z in counts:
        print( '{}.{} = {} (mod 2)'.format(b, z, bdotz(b,z)) )

In [41]:
runSimonAlgorithm(50)

Bitmap: 01001100000000000000000000000000000000000000000000
[50] Time: 0.0669
{'00111111110100110111100001100111110111000101100000': 1, '11101001111100011111111110000010010100110001000100': 1, '01100111101000110011001010010101000100111110010110': 1, '11001001000011100101000001010101110011110100001111': 1, '00011101011010110000111001010110110010111101000000': 1, '11010111011001100101100111010101101010010001100110': 1, '11110100011111111010100001011000001100100111101011': 1, '01101010111001000010101110100000100000110101001100': 1, '10001111010100111010011100101110110110110101010101': 1, '01001011111101111001101101100110100110111110110000': 1, '00011110011111011101110000101100001011100111101110': 1, '11101000000010001010100111100000001010001000101111': 1, '10110010100101001000100010100110011111000001101001': 1, '00000000100100111011100100111010111101110111000011': 1, '01000111110001001011010010000001111110100101011101': 1, '11011000101101110011000110010011101000001011111101': 1, '010101110